In [4]:
from transformers import pipeline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn

from sklearn.metrics import confusion_matrix, f1_score, roc_auc_score
from sklearn.model_selection import train_test_split

In [5]:
clf = pipeline("sentiment-analysis","distilbert-base-uncased-finetuned-sst-2-english")

In [7]:
type(clf)

transformers.pipelines.text_classification.TextClassificationPipeline

In [10]:
clf(["this is a good movie",
     "this is a bad movie",
     "i don't know what to say about this movie"])

[{'label': 'POSITIVE', 'score': 0.9998646974563599},
 {'label': 'NEGATIVE', 'score': 0.9997872710227966},
 {'label': 'NEGATIVE', 'score': 0.9992220401763916}]

In [1]:
import torch

In [2]:
torch.cuda.is_available()

False